### Notebook responsável por gerenciar as operações com o banco de dados

In [9]:
import datetime
from sqlalchemy import create_engine

In [10]:
user='postgres'
passwd='postgres'
host='postgresql'
port='5432'
db='cartola'
url = 'postgresql://{user}:{passwd}@{host}:{port}/{db}'.format(user=user, passwd=passwd, host=host, port=port, db=db)

In [11]:
def create_database_session():
    engine = create_engine(url)
    return engine

In [13]:
def create_table_if_not_exists(engine):
    engine.execute('''
        CREATE TABLE IF NOT EXISTS match (
            match_id integer, 
            turn smallint,
            home_id smallint,
            visitor_id smallint,
            date varchar(20),
            timestamp bigint,
            local varchar(80),
            valid boolean,
            home_goal smallint,
            visitor_goal smallint,
            year smallint
        )'''
    )

In [24]:
def alter_table_if_column_year_not_exists(engine):
    engine.execute('ALTER TABLE match ADD COLUMN IF NOT EXISTS year INTEGER NOT NULL DEFAULT 2021')

In [ ]:
def erase_data(engine):
    year = datetime.date.today().year
    engine.execute('delete from match where year = {}'.format(year))

In [33]:
def save_data(engine, table_name, dataframe):
    dataframe.to_sql(table_name, con=engine, index=False, if_exists='replace')

In [16]:
def count_data(engine):
    return engine.execute('SELECT count(*) FROM match')

In [28]:
def start_database():
    engine = create_database_session()
    create_table_if_not_exists(engine)
    alter_table_if_column_year_not_exists(engine)
    erase_data(engine)
    return engine